# Progetto Foundations of Computer Science


### Giuseppe Monea - 850432

In [1]:
import pandas as pd 
import numpy as np
from itertools import chain
import time
import datetime
import random
#si rimuove il limite di colonne visualizzabili
pd.set_option('display.max_columns', None)

## 1. Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.

In [2]:
df = pd.read_csv('loans_lenders.csv')

In [3]:
df.head(5)

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


In [4]:
def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

# calculate lengths of splits
lens = df['lenders'].str.split(',').map(len)

# create new dataframe, repeating or chaining as appropriate
loan_lenders = pd.DataFrame({'loan_id': np.repeat(df['loan_id'], lens),
                    'lenders': chainer(df['lenders'])})


In [5]:
loan_lenders.head()

,loan_id,lenders
0,483693,muc888
0,483693,sam4326
0,483693,camaran3922
0,483693,lachheb1865
0,483693,rebecca3499


## 2. For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

In [6]:
loans=pd.read_csv('loans.csv')

In [7]:
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,to buy additional inventories for her sari-sa...,PH,Philippines,Ozamiz - Ozamiz City,shared,0.1,PHP,126.0,2014-01-15 02:23:45.000 +0000,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,"To buy American clothing such as blouses, shir...",HN,Honduras,"La Lopez, Choloma",shared,0.1,HNL,201.0,2014-01-14 20:23:20.000 +0000,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,"to buy stock of rice, sugar and flour",PK,Pakistan,Lala Musa,shared,0.1,PKR,245.0,2014-01-16 11:32:58.000 +0000,2014-02-15 21:10:05.000 +0000,2014-01-09 08:00:00.000 +0000,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,"to buy cucumber and tomato seeds, as well as f...",KG,Kyrgyzstan,"Aravan village, Osh region",shared,0.1,KGS,171.0,2014-01-20 09:59:48.000 +0000,2014-02-21 03:10:02.000 +0000,2014-01-17 08:00:00.000 +0000,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,to buy organic fertilizer and agrochemical pr...,PH,Philippines,"Baleleng, Sto. Thomas, Isabela",shared,0.1,PHP,123.0,2014-01-14 05:46:21.000 +0000,2014-02-13 06:10:02.000 +0000,2013-12-17 08:00:00.000 +0000,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner


In [8]:
loans['planned_expiration_time'] = pd.to_datetime(loans['planned_expiration_time'])
loans['disburse_time'] = pd.to_datetime(loans['disburse_time'])
#si calcola la differenza tra le date
loans['duration'] = (loans['planned_expiration_time'] - loans['disburse_time']).apply(lambda x: x.days)

loans['duration']

0          53.0
1          96.0
2          37.0
3          34.0
4          57.0
           ... 
1419602    39.0
1419603    39.0
1419604    51.0
1419605    63.0
1419606    61.0
Name: duration, Length: 1419607, dtype: float64

## 3. Find the lenders that have funded at least twice.

In [9]:
funder = pd.merge(loans['loan_id'].loc[loans['status']=='funded'],loan_lenders,on='loan_id')
funder["lenders"].value_counts().loc[lambda x: x >= 2]

 gooddogg1           142349
 trolltech4460       139191
 gmct                122156
 nms                  93283
 themissionbeltco     70151
                      ...  
 petra6141                2
 julie9729                2
 dawn3289                 2
 cyndi9193                2
matthew17749014           2
Name: lenders, Length: 993782, dtype: int64

## 4. For each country, compute how many loans have involved that country as borrowers.

In [10]:
loans['country_name'].value_counts()

Philippines         285336
Kenya               143699
Peru                 86000
Cambodia             79701
El Salvador          64037
                     ...  
Mauritania               1
Canada                   1
Botswana                 1
Uruguay                  1
Papua New Guinea         1
Name: country_name, Length: 96, dtype: int64

## 5. For each country, compute the overall amount of money borrowed.

In [11]:
loans.groupby('country_name')['loan_amount'].sum()

country_name
Afghanistan        1967950.0
Albania            4307350.0
Armenia           22950475.0
Azerbaijan        14784625.0
Belize              150175.0
                     ...    
Vietnam           24681100.0
Virgin Islands       10000.0
Yemen              3444000.0
Zambia             1978975.0
Zimbabwe           5851875.0
Name: loan_amount, Length: 96, dtype: float64

## 6. Like the previous point, but expressed as a percentage of the overall amount lent.

In [12]:
100*loans.groupby('country_name')['loan_amount'].sum()/loans['loan_amount'].sum()

country_name
Afghanistan       0.166573
Albania           0.364586
Armenia           1.942589
Azerbaijan        1.251410
Belize            0.012711
                    ...   
Vietnam           2.089074
Virgin Islands    0.000846
Yemen             0.291509
Zambia            0.167506
Zimbabwe          0.495318
Name: loan_amount, Length: 96, dtype: float64

## 7. Like the three previous points, but split for each year (with respect to disburse time).

In [13]:
#a. 4. For each country, compute how many loans have involved that country as borrowers.
loans.groupby([loans['country_name'],loans['disburse_time'].dt.year])['country_name'].value_counts()

country_name  disburse_time  country_name
Afghanistan   2007.0         Afghanistan      408
              2008.0         Afghanistan      370
              2009.0         Afghanistan      678
              2010.0         Afghanistan      632
              2011.0         Afghanistan      247
                                             ... 
Zimbabwe      2013.0         Zimbabwe         426
              2014.0         Zimbabwe        2078
              2015.0         Zimbabwe         600
              2016.0         Zimbabwe         808
              2017.0         Zimbabwe        1079
Name: country_name, Length: 748, dtype: int64

In [14]:
#b. 5. For each country, compute the overall amount of money borrowed.
loans.groupby([loans['country_name'],loans['disburse_time'].dt.year])['loan_amount'].sum()

country_name  disburse_time
Afghanistan   2007.0            194975.0
              2008.0            365375.0
              2009.0            585125.0
              2010.0            563350.0
              2011.0            245125.0
                                 ...    
Zimbabwe      2013.0            678525.0
              2014.0           1311575.0
              2015.0            723625.0
              2016.0            788600.0
              2017.0           1237600.0
Name: loan_amount, Length: 748, dtype: float64

In [15]:
#c. 6. Like the previous point, but expressed as a percentage of the overall amount lent.
100*loans.groupby([loans['country_name'],loans['disburse_time'].dt.year])['loan_amount'].sum()/loans['loan_amount'].sum()

country_name  disburse_time
Afghanistan   2007.0           0.016503
              2008.0           0.030926
              2009.0           0.049527
              2010.0           0.047683
              2011.0           0.020748
                                 ...   
Zimbabwe      2013.0           0.057432
              2014.0           0.111015
              2015.0           0.061250
              2016.0           0.066749
              2017.0           0.104754
Name: loan_amount, Length: 748, dtype: float64

## 8. For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

In [16]:
#inizio calcolando la quota prestata per ogni loan dai vari lenders
loans['lender_contribution']=loans['loan_amount']/loans['num_lenders_total']

lender_contribution=pd.merge(loan_lenders, loans[['loan_id','lender_contribution']],on ='loan_id')
#eseguo la query
lender_contribution.groupby('lenders')['lender_contribution'].sum()

lenders
 000               1485.309656
 00000             1249.947363
 0002              2201.180463
 00mike00            38.461538
 0101craign0101    2424.088932
                      ...     
zzanita              62.500000
zzcyna7269           44.285714
zzinnia              32.758621
zzmcfate           2033.750197
zzrvmf8538          112.820513
Name: lender_contribution, Length: 1639026, dtype: float64

## 9. For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

In [17]:
#carico il df per avere la nazionalità dei lender
lenders=pd.read_csv('lenders.csv')
lenders.head()

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0


In [18]:
#per individuare la nazionalità dei lenders si considera che essa sia distribuita con lo stesso andamento che è possibile 
#ricavare dalle righe in cui essa è presente

#si definisce una funzione per attribuire la nazionalità ai lenders

def nationality(country_code):
    random.seed(9876)
    results = list()
    
    values = pd.unique(country_code.dropna())
    results.append(values)
    
    probs = country_code.dropna().value_counts() / len(country_code.dropna())
    results.append(probs)
    
    index_rows_nan = country_code.index[country_code.isnull()]
    results.append(index_rows_nan)
    
    n = len(index_rows_nan)
    results.append(n)
    
    return (results)

In [19]:
results_function = nationality(lenders['country_code'])
assign_country_code = pd.Series(np.random.choice(results_function[0], p = results_function[1], size = results_function[3]), index = results_function[2])
lenders['country_code'].fillna(assign_country_code, inplace = True)

In [20]:
lender_contribution.head()

,loan_id,lenders,lender_contribution
0,483693,muc888,27.840909
1,483693,sam4326,27.840909
2,483693,camaran3922,27.840909
3,483693,lachheb1865,27.840909
4,483693,rebecca3499,27.840909


In [21]:
#ora è presente ovunque il country_code
lenders.head()

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,US,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,US,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,US,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,PL,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,US,1461301154,NaN,NaN,NaN,NaN,0


In [22]:
#trovo l'importo prestato per nazione
lent_merging=pd.merge(lenders,lender_contribution,left_on='permanent_name', right_on='lenders')
lent_per_country=lent_merging.groupby('country_code')['lender_contribution'].sum().reset_index(name = 'lent')

In [23]:
lent_merging.head()

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited,loan_id,lenders,lender_contribution
0,naresh2074,Naresh,NaN,NaN,US,1461303119,NaN,NaN,3.0,NaN,0,1059280,naresh2074,25.925926
1,naresh2074,Naresh,NaN,NaN,US,1461303119,NaN,NaN,3.0,NaN,0,1055468,naresh2074,32.894737
2,jinxian2217,jinxian,NaN,NaN,NO,1461302938,NaN,NaN,5.0,NaN,0,1292306,jinxian2217,45.161290
3,sreedevi9583,Sreedevi,NaN,NaN,US,1461303004,NaN,NaN,3.0,NaN,0,1052062,sreedevi9583,25.000000
4,nicole98153893,Nicole,NaN,NaN,US,1461303247,NaN,NaN,3.0,NaN,0,1137907,nicole98153893,28.571429


In [24]:
#trovo quanto è stato preso in prestito da ogni nazione
borrowed_by_country = loans.groupby('country_code')['loan_amount'].sum().reset_index(name = 'borrowed')

In [25]:
#creo la tabella che confronta prestiti presi e sottoscritti e la relativa differenza
difference_lent_borrowed=pd.merge(lent_per_country,borrowed_by_country,on='country_code').fillna(0).round(2)
difference_lent_borrowed['difference']= difference_lent_borrowed['lent'] - difference_lent_borrowed['borrowed']
difference_lent_borrowed

,country_code,lent,borrowed,difference
0,AF,5382.14,1967950.0,-1962567.86
1,AL,1602.43,4307350.0,-4305747.57
2,AM,3300.83,22950475.0,-22947174.17
3,AZ,376.05,14784625.0,-14784248.95
4,BA,4608.37,477250.0,-472641.63
...,...,...,...,...
88,XK,302.50,3083025.0,-3082722.50
89,YE,5150.65,3444000.0,-3438849.35
90,ZA,25539.37,1006525.0,-980985.63
91,ZM,1017.10,1978975.0,-1977957.90


## 10. Which country has the highest ratio between the difference computed at the previous point and the population?

In [26]:
country_stats = pd.read_csv('country_stats.csv')
country_stats.head()

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh


In [27]:
difference_lent_borrowed=pd.merge(difference_lent_borrowed,country_stats[['country_code', 'population','population_below_poverty_line']], on = 'country_code')


difference_lent_borrowed = difference_lent_borrowed.set_index('country_code')

(difference_lent_borrowed['difference'] / difference_lent_borrowed['population']).idxmax()

'CA'

## 11. Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

In [28]:
difference_lent_borrowed['population_not_below_poverty_line'] = 100 - difference_lent_borrowed['population_below_poverty_line']
(difference_lent_borrowed['difference'] / (difference_lent_borrowed['population_not_below_poverty_line'])*difference_lent_borrowed['population']/100).idxmax()

'CA'

## 12. For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

In [29]:
loans_12=loans[['loan_id','disburse_time','planned_expiration_time','duration','loan_amount']].dropna()

In [30]:
loans_12['planned_expiration_time'] = pd.to_datetime(loans_12['planned_expiration_time'])

In [31]:
loans_12['disburse_time_year']=loans_12['disburse_time'].dt.year
loans_12['planned_expiration_time_year']=loans_12['planned_expiration_time'].dt.year
loans_12['disburse_time']=loans_12['disburse_time'].dt.date
loans_12['planned_expiration_time']=loans_12['planned_expiration_time'].dt.date

In [32]:
loans_12

,loan_id,disburse_time,planned_expiration_time,duration,loan_amount,disburse_time_year,planned_expiration_time_year
0,657307,2013-12-22,2014-02-14,53.0,125.0,2013,2014
1,657259,2013-12-20,2014-03-26,96.0,400.0,2013,2014
2,658010,2014-01-09,2014-02-15,37.0,400.0,2014,2014
3,659347,2014-01-17,2014-02-21,34.0,625.0,2014,2014
4,656933,2013-12-17,2014-02-13,57.0,425.0,2013,2014
...,...,...,...,...,...,...,...
1419602,988180,2015-11-23,2016-01-02,39.0,400.0,2015,2016
1419603,988213,2015-11-24,2016-01-02,39.0,300.0,2015,2016
1419604,989109,2015-11-13,2016-01-03,51.0,2425.0,2015,2016
1419605,989143,2015-11-03,2016-01-05,63.0,100.0,2015,2016


In [33]:
loans_12['years']=loans_12.apply(lambda x: list(range(x['disburse_time'].year,(x['planned_expiration_time'].year+1))),axis=1)

In [34]:
loans_12

,loan_id,disburse_time,planned_expiration_time,duration,loan_amount,disburse_time_year,planned_expiration_time_year,years
0,657307,2013-12-22,2014-02-14,53.0,125.0,2013,2014,"[2013, 2014]"
1,657259,2013-12-20,2014-03-26,96.0,400.0,2013,2014,"[2013, 2014]"
2,658010,2014-01-09,2014-02-15,37.0,400.0,2014,2014,[2014]
3,659347,2014-01-17,2014-02-21,34.0,625.0,2014,2014,[2014]
4,656933,2013-12-17,2014-02-13,57.0,425.0,2013,2014,"[2013, 2014]"
...,...,...,...,...,...,...,...,...
1419602,988180,2015-11-23,2016-01-02,39.0,400.0,2015,2016,"[2015, 2016]"
1419603,988213,2015-11-24,2016-01-02,39.0,300.0,2015,2016,"[2015, 2016]"
1419604,989109,2015-11-13,2016-01-03,51.0,2425.0,2015,2016,"[2015, 2016]"
1419605,989143,2015-11-03,2016-01-05,63.0,100.0,2015,2016,"[2015, 2016]"


In [35]:
loans_12=loans_12.explode('years').reset_index(drop=True).dropna()
loans_12

,loan_id,disburse_time,planned_expiration_time,duration,loan_amount,disburse_time_year,planned_expiration_time_year,years
0,657307,2013-12-22,2014-02-14,53.0,125.0,2013,2014,2013
1,657307,2013-12-22,2014-02-14,53.0,125.0,2013,2014,2014
2,657259,2013-12-20,2014-03-26,96.0,400.0,2013,2014,2013
3,657259,2013-12-20,2014-03-26,96.0,400.0,2013,2014,2014
4,658010,2014-01-09,2014-02-15,37.0,400.0,2014,2014,2014
...,...,...,...,...,...,...,...,...
1196549,989109,2015-11-13,2016-01-03,51.0,2425.0,2015,2016,2016
1196550,989143,2015-11-03,2016-01-05,63.0,100.0,2015,2016,2015
1196551,989143,2015-11-03,2016-01-05,63.0,100.0,2015,2016,2016
1196552,989240,2015-11-03,2016-01-03,61.0,175.0,2015,2016,2015


In [36]:
loans_12 = loans_12[loans_12.duration != 0]

In [37]:
def amount_per_year(years, disburse_time, planned_expiration_time, disburse_time_year, planned_expiration_time_year, loan_amount, duration):
    
    #se planned_expiration_time precede disburse_time considero l'errore come di inserimento nel dato e li inverto
    if (disburse_time.year < planned_expiration_time.year):
        start_date = disburse_time
        end_date = planned_expiration_time
    else:
        start_date = planned_expiration_time
        end_date = disburse_time
    
    amount_per_day = loan_amount/duration
    
    #trovo quanti giorni per l'anno considerato è attivo il prestito
    if (disburse_time_year == planned_expiration_time_year):
        return loan_amount
    
    elif years == start_date.year:
        #da data inizio a fine anno
        interval = (datetime.date(years, 12, 31)-disburse_time).days
        return amount_per_day*interval
    
    elif years > start_date.year:
        #intervallo da inizio anno  a prima tra fine anno e fine prestito
        if planned_expiration_time.year == years:
            interval = (planned_expiration_time-datetime.date(years, 1, 1)).days
        else:
            interval = (datetime.date(years, 12, 31)-datetime.date(years, 1, 1)).days     
        return amount_per_day*interval
    
    

In [38]:
loans_12['amount_per_year']=loans_12.apply(lambda x: amount_per_year(x['years'],x['disburse_time'],x['planned_expiration_time'],x['disburse_time_year'],x['planned_expiration_time_year'],x['loan_amount'],x['duration']),axis=1)

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [39]:
loans_12.groupby(['years'],as_index=False)['amount_per_year'].sum().round(1)

,years,amount_per_year
0,2011,631678.3
1,2012,113199070.2
2,2013,124665237.0
3,2014,152554159.3
4,2015,159048482.9
5,2016,158278530.6
6,2017,171024983.5
7,2018,5421198.4
